## 아키텍처 구조만 작성

![workflow](/home/ansgyqja/chatbot/images/archtiecture_flow.png)


## 그래프 생성절차

1. State 정의
2. 노드 정의
3. 그래프 정의
4. 그래프 컴파일
5. 그래프 시각화



In [1]:
from typing import TypedDict, Annotated, List, Dict
from langchain_core.documents import Document
from langgraph.graph.message import add_messages
import operator
from langchain_core.tools import tool
from module.get_naver_news import get_mcp_server_response
from module.get_naver_news import get_news_origin
from module.get_local_pdf import rag_pdf
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END

from langchain_teddynote.graphs import visualize_graph


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# State 정의
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    context: Annotated[List[Document], operator.add]
    answer: Annotated[List[Document], operator.add]
    question: Annotated[str, "user question"]
    argument_question:Annotated[list[str],operator.add]
    binary_score: Annotated[str, "binary score yes or no"]

## 멀티 쿼리 확장 

In [3]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.(5개의 질문을 생성하도록 프롬프트를 작성하였습니다)
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant. 
Your task is to generate three different versions of the given user question to retrieve relevant documents from a vector database. 
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION: 
{question}

#Answer in Korean:
"""
)

# 언어 모델 인스턴스를 생성합니다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# LLMChain을 생성합니다.
custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

# 질문을 정의합니다.
# question =  "내부 정보 활용하여 삼성전자 개발중 AI 모델 알려줘"
question = '안녕'

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = custom_multiquery_chain.invoke(question)
print(multi_queries)

안녕하세요  
안녕하십니까  
안녕, 어떻게 지내세요?  


In [ ]:
# 도구 생성
@tool
async def search_naver_news(query: str) -> List[Dict[str, str]]:
    """Search naver News by input keyword"""
    mcp_response = await get_mcp_server_response(query)
    return get_news_origin(mcp_response)

# @tool
# def search_naver_news(query: str) -> List[Dict[str, str]]:
#     """Search naver News by input keyword"""
    
#     return query


@tool
def search_local_data(query: str) -> List[Dict[str, str]]:
    """Search our company pdf file infomation by input keyword"""
    news_tool = rag_pdf()
    return news_tool.search_by_keyword(query)

@tool
def search_db_data(query: str) -> List[Dict[str, str]]:
    """Search our company database infomation by input keyword"""

    return query

In [5]:
tools = [search_naver_news, search_local_data,search_db_data]

# ToolNode 초기화
tool_node = ToolNode(tools)

model_with_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [6]:
print(model_with_tools)

bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fa21bf99f10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fa21ad7ad90>, root_client=<openai.OpenAI object at 0x7fa21bf89a10>, root_async_client=<openai.AsyncOpenAI object at 0x7fa21c87eb10>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'search_naver_news', 'description': 'Search naver News by input keyword', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'search_local_data', 'description': 'Search our company pdf file infomation by input keyword', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'search_db_data', 'description': 'Search our c

In [7]:
# LLM 모델을 사용하여 메시지 처리 및 응답 생성, 도구 호출이 포함된 응답 반환
def find_tools(state: GraphState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

def rewrite_query(state: GraphState) -> GraphState:
    # Query Transform: 쿼리 재작성
    documents = "검색된 문서"
    return GraphState(context=documents)

def sum_up(state: GraphState) -> GraphState:
    # sum_up: 결과 종합
    answer = "종합된 답변"
    return GraphState(answer=answer)

def relevance_check(state: GraphState) -> GraphState:
    # Relevance Check: 관련성 확인
    binary_score = "Relevance Score"
    return GraphState(binary_score=binary_score)


def decision(state: GraphState) -> GraphState:
    # 의사결정
    decision = "결정"
    # 로직을 추가할 수 가 있고요.

    if state["binary_score"] == "yes":
        return "종료"
    else:
        return "재검색"

In [8]:


# 메시지 상태 기반 워크플로우 그래프 초기화
workflow = StateGraph(GraphState)

# 에이전트와 도구 노드 정의 및 워크플로우 그래프에 추가
workflow.add_node("find_tools", find_tools)
workflow.add_node("tools", tool_node)
workflow.add_node("sum_up", sum_up)
# workflow.add_node("relevance_check", relevance_check)
# workflow.add_node("decision", decision)
# workflow.add_node("rewrite_query", rewrite_query)


workflow.add_edge(START, "find_tools")

# 에이전트 노드에서 조건부 분기 설정, 도구 노드 또는 종료 지점으로 연결
workflow.add_conditional_edges("find_tools", tools_condition)

workflow.add_edge("tools", "sum_up")

workflow.add_edge('sum_up', END)

# workflow.add_edge("tools", "sum_up")

# workflow.add_edge("find_tools", "sum_up")

# workflow.add_edge("sum_up", "relevance_check")

# workflow.add_conditional_edges(
#     "relevance_check",  # 관련성 체크 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
#     decision,
#     {
#         "재검색": "rewrite_query",  # 관련성이 있으면 종료합니다.
#         "종료": END,  # 관련성 체크 결과가 모호하다면 다시 답변을 생성합니다.
#     },
# )

# workflow.add_edge("rewrite_query", 'find_tools')


# workflow.add_edge("decision", END)

# 정의된 워크플로우 그래프 컴파일 및 실행 가능한 애플리케이션 생성
graph = workflow.compile()

# result = await graph.ainvoke( {"messages": [("human",multi_queries)]})
from langchain_teddynote.graphs import visualize_graph

visualize_graph(graph)

그래프 시각화 실패 (추가 종속성 필요): Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`
ASCII로 그래프 표시:
        +-----------+     
        | __start__ |     
        +-----------+     
               *          
               *          
               *          
        +------------+    
        | find_tools |    
        +------------+    
          .        ..     
        ..           .    
       .              ..  
+-------+               . 
| tools |               . 
+-------+               . 
     *                  . 
     *                  . 
     *                  . 
+--------+              . 
| sum_up |            ..  
+--------+

In [9]:
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import Image, display

# Mermaid 그래프를 그리는 객체(graph)는 사용자가 직접 정의해야 합니다.
# 예제에서는 graph.get_graph()로 그래프 객체를 얻었다고 가정합니다.

# 해당 객체에서 draw_mermaid_png 메서드 호출 시 draw_method에 PYPPETEER 지정
img = graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)

# IPython 환경(Jupyter Notebook 등)에서 이미지를 화면에 표시
display(Image(img))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
print(result['messages'])

In [ ]:
# print(result['messages'][0])
print(result['messages'][1])

In [ ]:



def query_argument(state: GraphState)-> GraphState:
    # 사용자 query 증강 3개
    query=[]
    return GraphState(argument_question=query)

def tool_decision_bot(state: GraphState)->GraphState:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

